In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random

### 1. Read original data from kaggle

In [2]:
df = pd.read_csv('customer_shopping_data.csv', parse_dates=['invoice_date'], dayfirst=True)
df = df.sort_values(['invoice_date']).reset_index().drop(['index'], axis=1)
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park
...,...,...,...,...,...,...,...,...,...,...
99452,I218048,C389917,Female,66,Clothing,5,1500.40,Cash,2023-03-08,Mall of Istanbul
99453,I268432,C175797,Female,37,Food & Beverage,3,15.69,Cash,2023-03-08,Kanyon
99454,I200744,C224026,Female,59,Cosmetics,5,203.30,Credit Card,2023-03-08,Kanyon
99455,I124836,C265661,Female,46,Shoes,3,1800.51,Credit Card,2023-03-08,Metrocity


In [5]:
df.describe()

,age,quantity,price,invoice_date
count,99457.000000,99457.000000,99457.000000,99457
mean,43.427089,3.003429,689.256321,2022-02-04 02:46:59.783424256
min,18.000000,1.000000,5.230000,2021-01-01 00:00:00
25%,30.000000,2.000000,45.450000,2021-07-19 00:00:00
50%,43.000000,3.000000,203.300000,2022-02-05 00:00:00
75%,56.000000,4.000000,1200.320000,2022-08-22 00:00:00
max,69.000000,5.000000,5250.000000,2023-03-08 00:00:00
std,14.990054,1.413025,941.184567,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   invoice_no      99457 non-null  object        
 1   customer_id     99457 non-null  object        
 2   gender          99457 non-null  object        
 3   age             99457 non-null  int64         
 4   category        99457 non-null  object        
 5   quantity        99457 non-null  int64         
 6   price           99457 non-null  float64       
 7   payment_method  99457 non-null  object        
 8   invoice_date    99457 non-null  datetime64[ns]
 9   shopping_mall   99457 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 7.6+ MB


#### 2. Add new rows to dataframe

In [7]:
from collections import defaultdict

category_prices = defaultdict(list)

for category in df['category'].unique():
    category_prices[category] = df[df['category'] == category]['price'].unique()

category_prices

defaultdict(list,
            {'Food & Beverage': array([26.15, 20.92, 15.69, 10.46,  5.23]),
             'Clothing': array([1200.32,  900.24,  300.08, 1500.4 ,  600.16]),
             'Shoes': array([1800.51,  600.17, 2400.68, 3000.85, 1200.34]),
             'Souvenir': array([23.46, 11.73, 58.65, 35.19, 46.92]),
             'Cosmetics': array([ 81.32, 121.98, 203.3 ,  40.66, 162.64]),
             'Toys': array([179.2 ,  71.68, 107.52, 143.36,  35.84]),
             'Books': array([60.6 , 75.75, 45.45, 30.3 , 15.15]),
             'Technology': array([1050., 5250., 2100., 4200., 3150.])})

In [9]:
def generate_date_fast(df_sales, start_date, end_date, avg_transactions_per_day):
    date_series = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Sử dụng phân phối Poisson để mô phỏng số lượng giao dịch ngẫu nhiên nhưng có mức trung bình cố định
    # 125 là mức trung bình giao dịch/ngày bạn cần
    daily_counts = np.random.poisson(lam=avg_transactions_per_day, size=len(date_series))
    
    dates = []
    for date, count in zip(date_series, daily_counts):
        dates.extend([date] * count)

    num_rows = sum(daily_counts)

    from collections import defaultdict
    category_prices = defaultdict(list) 
    for cat in df_sales['category'].unique():
        category_prices[cat] = df_sales[df_sales['category'] == cat]['price'].unique()

    categories = np.random.choice(df_sales['category'].unique(), size=num_rows)
    prices = np.array([np.random.choice(category_prices[cat]) for cat in categories])

    data = pd.DataFrame({
        'invoice_no': ['I' + str(np.random.randint(100000, 999999)) for _ in range(num_rows)],
        'customer_id': ['C' + str(np.random.randint(100000, 999999)) for _ in range(num_rows)],
        'gender': np.random.choice(df_sales['gender'], size=num_rows),
        'age': np.random.randint(18, 70, size=num_rows),
        'category': categories,
        'quantity': np.random.randint(1, 6, size=num_rows),
        'price': prices,
        'payment_method': np.random.choice(df_sales['payment_method'].unique(), size=num_rows),
        'invoice_date': dates,
        'shopping_mall': np.random.choice(df_sales['shopping_mall'].unique(), size=num_rows)
    })

    return data

# # Sinh 40k rows
added_data = generate_date_fast(df_sales=df, start_date=datetime(2023,3,8), end_date=datetime(2024,1,15), avg_transactions_per_day=150)

# concat data gốc với added_data
df = pd.concat([df, added_data]).reset_index(drop=True)
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park
...,...,...,...,...,...,...,...,...,...,...
146666,I986197,C793506,Female,28,Food & Beverage,5,26.15,Credit Card,2024-01-15,Zorlu Center
146667,I855732,C417341,Male,63,Books,1,60.60,Debit Card,2024-01-15,Cevahir AVM
146668,I469064,C794451,Female,31,Technology,5,4200.00,Debit Card,2024-01-15,Metropol AVM
146669,I997957,C179229,Male,18,Toys,5,71.68,Credit Card,2024-01-15,Forum Istanbul


### 3. Add missing data and outliers

In [10]:
def add_missing_nans(data, nan_percentage=0.01, outlier_percentage=0.001):
    modified_data = data.copy()

    # Calculate the number of rows to add outliers and NaN values
    num_rows = len(modified_data)
    num_outliers = int(num_rows * outlier_percentage)
    num_nans = int(num_rows * nan_percentage)


    np.random.seed(2025)
    outlier_indices = np.random.choice(num_rows, num_outliers, replace=False)
    modified_data.loc[
        outlier_indices, "price"
    ] *= 3  # Increase sales by a factor to create outliers

    # Add NaN values to the 'sales' column
    nan_indices = np.random.choice(num_rows, num_nans, replace=False)
    modified_data.loc[nan_indices, "price"] = np.nan

    return modified_data

In [11]:
modified_data = add_missing_nans(df)
modified_data

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity
1,I313757,C438201,Female,43,Clothing,4,1200.32,Credit Card,2021-01-01,Mall of Istanbul
2,I836951,C382440,Male,54,Clothing,3,900.24,Cash,2021-01-01,Metrocity
3,I159235,C312481,Male,58,Food & Beverage,4,20.92,Credit Card,2021-01-01,Mall of Istanbul
4,I115941,C105769,Male,34,Clothing,1,300.08,Cash,2021-01-01,Istinye Park
...,...,...,...,...,...,...,...,...,...,...
146666,I986197,C793506,Female,28,Food & Beverage,5,26.15,Credit Card,2024-01-15,Zorlu Center
146667,I855732,C417341,Male,63,Books,1,60.60,Debit Card,2024-01-15,Cevahir AVM
146668,I469064,C794451,Female,31,Technology,5,4200.00,Debit Card,2024-01-15,Metropol AVM
146669,I997957,C179229,Male,18,Toys,5,71.68,Credit Card,2024-01-15,Forum Istanbul


In [12]:
modified_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146671 entries, 0 to 146670
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   invoice_no      146671 non-null  object        
 1   customer_id     146671 non-null  object        
 2   gender          146671 non-null  object        
 3   age             146671 non-null  int64         
 4   category        146671 non-null  object        
 5   quantity        146671 non-null  int64         
 6   price           145205 non-null  float64       
 7   payment_method  146671 non-null  object        
 8   invoice_date    146671 non-null  datetime64[ns]
 9   shopping_mall   146671 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 11.2+ MB


In [14]:
modified_data.to_csv('sales_data_modified.csv', index=False)

### 4. Preparing weather data in turkey

In [ ]:
""" 
< 2 mm → mưa nhỏ
2–10 mm → mưa vừa
10 mm → mưa to
50 mm → mưa rất lớn
"""

import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 41.0082,
	"longitude": 28.9784,
	"start_date": "2021-01-01",
	"end_date": "2024-01-16",
	"daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum"],
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe

Coordinates: 41.01932907104492°N 29.01569175720215°E
Elevation: 36.0 m asl
Timezone difference to GMT+0: 0s


,date,temperature_2m_max,temperature_2m_min,rain_sum
0,2021-01-01 00:00:00+00:00,13.806000,7.756000,4.200000
1,2021-01-02 00:00:00+00:00,15.906000,5.656000,0.000000
2,2021-01-03 00:00:00+00:00,14.456000,8.256001,0.200000
3,2021-01-04 00:00:00+00:00,15.556000,8.756001,0.000000
4,2021-01-05 00:00:00+00:00,14.756001,7.656000,1.600000
...,...,...,...,...
1106,2024-01-12 00:00:00+00:00,6.756000,-0.894000,0.500000
1107,2024-01-13 00:00:00+00:00,3.956000,-1.644000,0.000000
1108,2024-01-14 00:00:00+00:00,7.456000,-2.244000,0.000000
1109,2024-01-15 00:00:00+00:00,11.806000,6.556000,0.200000


In [ ]:
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   date                1111 non-null   datetime64[ns, UTC]
 1   temperature_2m_max  1111 non-null   float32            
 2   temperature_2m_min  1111 non-null   float32            
 3   rain_sum            1111 non-null   float32            
dtypes: datetime64[ns, UTC](1), float32(3)
memory usage: 21.8 KB


In [ ]:
daily_dataframe.describe()

,temperature_2m_max,temperature_2m_min,rain_sum
count,1111.000000,1111.000000,1111.000000
mean,18.993759,11.731968,2.112961
std,7.890963,6.806037,5.201321
min,0.906000,-6.444000,0.000000
25%,13.006001,6.556000,0.000000
50%,18.806000,11.506001,0.000000
75%,25.556000,17.606001,1.500000
max,38.806000,24.556000,57.700001


In [ ]:
daily_dataframe.to_csv('weather_data.csv', index=False)